In [25]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd
import os
import csv
import datetime as dt
# import openweathermapy as ow # API call to get city long lat from city name

from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, Float

import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

# from opencage.geocoder import OpenCageGeocode # used to get long and lat of given city name
from math import radians, sin, cos, asin, sqrt # used to calculate distance of room and city longs and lats

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# Google API Key
from config import gkey


from flask import Flask, jsonify

In [4]:
results = requests.get('https://public.opendatasoft.com/api/records/1.0/download/?dataset=air-bnb-listings&rows=100&format=json').json()


In [5]:
pprint(results)

[{'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 347,
             'calculated_host_listings_count': 74,
             'city': 'New-orleans',
             'column_10': 1003,
             'column_19': 'United states',
             'column_20': 'United states, New-orleans, Central Business '
                          'District',
             'coordinates': [29.952860923402337, -90.07131139281506],
             'host_id': 114353388,
             'id': 19051656,
             'last_review': '2017-10-19',
             'minimum_nights': 1,
             'name': 'Gorgeous Stay Alfred on North Carondelet Street',
             'neighbourhood': 'Central Business District',
             'number_of_reviews': 1,
             'reviews_per_month': 0.05,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.07131139281506, 29.952860923402337],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:0

             'reviews_per_month': 4.64,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.07246907676978, 29.95481744177011],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': 'f8885a54659989c9bfe9c56c96e494374123f6ef'},
 {'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 151,
             'calculated_host_listings_count': 1,
             'city': 'New-orleans',
             'column_10': 70,
             'column_19': 'United states',
             'column_20': 'United states, New-orleans, Tulane - Gravier',
             'coordinates': [29.966453274657894, -90.08167360323176],
             'host_id': 169210168,
             'id': 22869975,
             'last_review': '2019-04-27',
             'minimum_nights': 2,
             'name': 'Creole Cottage & Cafe: 6 minutes to anywhere!',
             'neighbourhood': 'Tulane - Gravier',
             'number_of_re

             'column_20': 'United states, New-orleans, French Quarter',
             'coordinates': [29.95164959814229, -90.06579923497043],
             'host_id': 88384322,
             'id': 24756994,
             'last_review': '2019-04-14',
             'minimum_nights': 1,
             'name': 'Authentic 1BR/1BA in Historic New Orleans 402',
             'neighbourhood': 'French Quarter',
             'number_of_reviews': 34,
             'reviews_per_month': 2.79,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.06579923497043, 29.95164959814229],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': '1a24c0b37898c58547ea0f0c3e8d6080faf88bf8'},
 {'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 72,
             'calculated_host_listings_count': 5,
             'city': 'New-orleans',
             'column_10': 92,
             'column_19': 'United stat

             'column_10': 499,
             'column_19': 'United states',
             'column_20': 'United states, New-orleans, Lower Garden District',
             'coordinates': [29.930799079858346, -90.07509737599439],
             'host_id': 37614659,
             'id': 29234907,
             'last_review': '2019-04-07',
             'minimum_nights': 3,
             'name': 'Casabianca: Elegant Flat in Historic Magazine Row',
             'neighbourhood': 'Lower Garden District',
             'number_of_reviews': 15,
             'reviews_per_month': 2.74,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.07509737599439, 29.930799079858346],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': 'ae1b4133d6de9b32fe1b0d36014a08ef36f669c3'},
 {'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 319,
             'calculated_host_listings_count': 13,
        

In [6]:
results = [r["fields"] for r in results]
results

[{'updated_date': '2019-05-05',
  'reviews_per_month': 0.05,
  'calculated_host_listings_count': 74,
  'city': 'New-orleans',
  'minimum_nights': 1,
  'coordinates': [29.952860923402337, -90.07131139281506],
  'room_type': 'Entire home/apt',
  'number_of_reviews': 1,
  'column_10': 1003,
  'neighbourhood': 'Central Business District',
  'last_review': '2017-10-19',
  'host_id': 114353388,
  'column_19': 'United states',
  'availability_365': 347,
  'id': 19051656,
  'column_20': 'United states, New-orleans, Central Business District',
  'name': 'Gorgeous Stay Alfred on North Carondelet Street'},
 {'updated_date': '2019-05-05',
  'reviews_per_month': 4.84,
  'calculated_host_listings_count': 1,
  'city': 'New-orleans',
  'minimum_nights': 1,
  'coordinates': [29.987034337344596, -89.94496848089439],
  'room_type': 'Entire home/apt',
  'number_of_reviews': 20,
  'column_10': 74,
  'neighbourhood': 'Viavant - Venetian Isles',
  'last_review': '2019-04-28',
  'host_id': 61310992,
  'column

In [7]:
results_df = pd.DataFrame(results)
results_df.head()

,availability_365,calculated_host_listings_count,city,column_10,column_19,column_20,coordinates,host_id,id,last_review,minimum_nights,name,neighbourhood,number_of_reviews,reviews_per_month,room_type,updated_date
0,347,74,New-orleans,1003,United states,"United states, New-orleans, Central Business D...","[29.952860923402337, -90.07131139281506]",114353388,19051656,2017-10-19,1,Gorgeous Stay Alfred on North Carondelet Street,Central Business District,1,0.05,Entire home/apt,2019-05-05
1,302,1,New-orleans,74,United states,"United states, New-orleans, Viavant - Venetian...","[29.987034337344596, -89.94496848089439]",61310992,19079967,2019-04-28,1,Cozy New Orleans Efficiency On The Water,Viavant - Venetian Isles,20,4.84,Entire home/apt,2019-05-05
2,206,1,New-orleans,175,United states,"United states, New-orleans, Marlyville - Fonta...","[29.954771004938046, -90.11994216301255]",22171971,19123547,2019-04-22,2,Townhome on the Streetcar Line,Marlyville - Fontainbleau,74,3.65,Entire home/apt,2019-05-05
3,0,2,New-orleans,86,United states,"United states, New-orleans, Lower Garden District","[29.93808917158898, -90.07359074429189]",710534,19245713,2019-04-29,2,Stylish 1BR Shotgun in Lower Garden District,Lower Garden District,89,3.90,Entire home/apt,2019-05-05
4,25,4,New-orleans,84,United states,"United states, New-orleans, Seventh Ward","[29.971855482779855, -90.06922180405711]",2428040,19267950,2019-04-27,1,Relaxing 3BR Balcony Apartment,Seventh Ward,22,1.08,Entire home/apt,2019-05-05


In [8]:
cleanData = results_df[[ 'column_10','id', 'coordinates', 'room_type', "city", 'column_19']]
cleanData.head()

,column_10,id,coordinates,room_type,city,column_19
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New-orleans,United states
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New-orleans,United states
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New-orleans,United states
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New-orleans,United states
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New-orleans,United states


In [9]:
cleanData = cleanData.rename(index=str, columns={"column_10":"Price",
                                                 "id":"Room_Id", 
                                                 "coordinates":"Coordinates",
                                                 "column_19":"Country", 
                                                 "room_type":"Room_Type",
                                                 "city":"City"})

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New-orleans,United states
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New-orleans,United states
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New-orleans,United states
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New-orleans,United states
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New-orleans,United states


In [10]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cleanData['Coordinates']:
   # Try to,
   try:
       # Split the row by comma and append
       # everything before the comma to lat
       lat.append(row[0])
       # Split the row by comma and append
       # everything after the comma to lon
       lon.append(row[1])
   # But if you get an error
   except:
       # append a missing value to lat
       lat.append(np.NaN)
       # append a missing value to lon
       lon.append(np.NaN)

# Create two new columns from lat and lon
cleanData['Room_Lat'] = lat
cleanData['Room_Long'] = lon


In [11]:
# cleanData = cleanData.drop(columns=["Coordinates"])
# cleanData.head()

In [12]:
cleanData["Selection"] = cleanData["City"]+", "+cleanData["Country"]
# cleanData.head(5)

In [13]:
cleanData['City'] = cleanData['City'].str.replace('-',' ')

In [14]:
rooms_lat = []   # create empty lists
rooms_long = []
cities_lat = [] 
cities_long = []
calculated_distance = []

for index, row in cleanData.iterrows():
    room_lat1 = row['Room_Lat']
    room_long1 = row['Room_Long']
    
# Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(row, gkey)

#Run a request to endpoint and convert result to json
    cities_lat_long = requests.get(target_url).json()

    city_lat2 = cities_lat_long["results"][0]["geometry"]["location"]["lat"]
    city_long2 = cities_lat_long["results"][0]["geometry"]["location"]["lng"]
    
#CALCULATES DISTANCE OF ROOMS AND CITIES LAT AND LONGS
    def distance(room_long1, room_lat1, city_long2, city_lat2):
        room_long1, room_lat1, city_long2, city_lat2 = map(radians, [room_long1, room_lat1, city_long2, city_lat2])
        dlon = city_long2 - room_long1
        dlat = city_lat2 - room_lat1
        a = sin(dlat / 2) ** 2 + cos(room_lat1) * cos(city_lat2) * sin(dlon / 2) ** 2
        return 2 * 3958.756 * asin(sqrt(a)) #6371 km / 3958.756 for miles

    room_city_distance = distance(room_long1, room_lat1, city_long2, city_lat2)

    rooms_lat.append(room_lat1)
    rooms_long.append(room_long1)
    cities_lat.append(city_lat2)
    cities_long.append(city_long2)
    calculated_distance.append(room_city_distance)
    
print("rooms_lat:", rooms_lat)
print("rooms_long:", rooms_long)
print("cities_lat:", cities_lat)
print("cities_long:", cities_long)
print("calculated_distance:", calculated_distance)

# this site confirmed the calculation is correct:
# https://www.gpsvisualizer.com/calculators

rooms_lat: [29.952860923402337, 29.987034337344596, 29.954771004938046, 29.93808917158898, 29.971855482779855, 29.972984538974295, 29.971951208140464, 29.93437987023291, 29.964599422632258, 29.974925916272348, 29.95346693500604, 29.943894719621536, 29.965273097738862, 29.951575951595856, 29.942607102967557, 29.987014203087593, 29.91856449141886, 29.942308399098735, 29.954688525819865, 29.9220704128532, 29.94004919002642, 29.932277463287495, 29.921338251519295, 29.91511328072357, 29.930922419057197, 29.943018648245122, 29.95095051203267, 29.94914663076914, 29.95380634584867, 29.918962377885748, 29.979185081936954, 29.940708948399404, 29.936305437435777, 29.927262108077237, 30.02602247811368, 29.952597970461834, 29.971535274070852, 29.95481744177011, 29.966453274657894, 30.009158986004667, 29.970230809603674, 29.968016759310363, 29.920401959294892, 29.921850034364635, 29.96989321370255, 29.948666215409013, 29.95074496869807, 29.95196302240908, 29.953566444479158, 29.96827335556603, 30.01

In [15]:
# cleanData['City_Lat'] = cities_lat
# cleanData['City_Long'] = cities_long
cleanData['Calc_Distance'] = calculated_distance

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long,Selection,Calc_Distance
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New orleans,United states,29.952861,-90.071311,"New-orleans, United states",0.124734
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New orleans,United states,29.987034,-89.944968,"New-orleans, United states",7.972731
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New orleans,United states,29.954771,-90.119942,"New-orleans, United states",2.909339
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New orleans,United states,29.938089,-90.073591,"New-orleans, United states",0.905028
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New orleans,United states,29.971855,-90.069222,"New-orleans, United states",1.443071


In [20]:
cleanData= cleanData.drop(columns=["Coordinates"])
cleanData.head()

,Price,Room_Id,Room_Type,City,Country,Room_Lat,Room_Long,Selection,Calc_Distance
0,1003,19051656,Entire home/apt,New orleans,United states,29.952861,-90.071311,"New-orleans, United states",0.124734
1,74,19079967,Entire home/apt,New orleans,United states,29.987034,-89.944968,"New-orleans, United states",7.972731
2,175,19123547,Entire home/apt,New orleans,United states,29.954771,-90.119942,"New-orleans, United states",2.909339
3,86,19245713,Entire home/apt,New orleans,United states,29.938089,-90.073591,"New-orleans, United states",0.905028
4,84,19267950,Entire home/apt,New orleans,United states,29.971855,-90.069222,"New-orleans, United states",1.443071


In [22]:
# #Export city names to csv
# cleanData.to_csv('../data.csv', index=False)

# # Import cities file as DataFrame
# cleanData_df = pd.read_csv("../data.csv")
# cleanData_df.head()

In [ ]:
# this site confirmed the calculation is correct:
# https://www.gpsvisualizer.com/calculators

In [35]:
cleanData = cleanData.rename(index=str, columns={"Room_Lat":"Latitude",
                                                 "Room_Long":"Longitude", 
                                                 "Calc_Distance":"Distance"})

cleanData.head()

,Price,Room_Id,Room_Type,City,Country,Latitude,Longitude,Selection,Distance
0,1003,19051656,Entire home/apt,New orleans,United states,29.952861,-90.071311,"New-orleans, United states",0.124734
1,74,19079967,Entire home/apt,New orleans,United states,29.987034,-89.944968,"New-orleans, United states",7.972731
2,175,19123547,Entire home/apt,New orleans,United states,29.954771,-90.119942,"New-orleans, United states",2.909339
3,86,19245713,Entire home/apt,New orleans,United states,29.938089,-90.073591,"New-orleans, United states",0.905028
4,84,19267950,Entire home/apt,New orleans,United states,29.971855,-90.069222,"New-orleans, United states",1.443071


In [42]:
conn_str = "root:Fe529534@localhost/airbnb_data"
engine = create_engine(f'mysql://{conn_str}')

if not database_exists(engine.url):
    create_database(engine.url)

In [43]:
engine.table_names()

[]

In [44]:
metadata = MetaData(engine)

listings = Table('listings', metadata,
                 Column('Price', Integer),
                 Column('Room_Id', String(10)),
                 Column('Room_Type', String(25)),
                 Column('City', String(50)),
                 Column('Country', String(50)),
                 Column('Latitude', Float),
                 Column('Longitude', Float),
                 Column('Selection', String(100)),
                 Column('Distance', Float))

metadata.create_all()
for _t in metadata.tables:
    print("Table: ",_t)

Table:  listings


In [45]:
cleanData.to_sql(name="listings", con=engine, if_exists="append",index=False)

In [46]:
cityList = pd.read_sql_query('select distinct Selection from listings', con=engine)
cityList.head

<bound method NDFrame.head of                     Selection
0  New-orleans, United states>

In [47]:
cities = Table('cities', metadata,
                 Column('Selection', String(100))
              )

metadata.create_all()
for _t in metadata.tables:
    print("Table: ",_t)

Table:  listings
Table:  cities
